In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score

import pytry

In [2]:
from sklearn.decomposition import PCA

class PatternInterpolationTrial(pytry.Trial):
    def params(self):
        self.param('number of neurons', n_neurons=2000),
        
    def evaluate(self, q): #function to fill in missing data points by interpolating
        data = pd.read_csv("pinsoro-2017-06-20-145454904791-small.csv", low_memory=False)
        
        x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
        labs = np.array(data.iloc[:,218]).astype(str) #array of labels (purple child annotations, engagement)
        a = []
        for i in range(x.shape[1]):
            y = pd.Series(x[:,i])
            z = y.interpolate(limit_direction='both')
            a.append(z)
        a = pd.DataFrame(a)
        a = a.dropna()
        a = np.array(a).T

        dt = 0.001
        
        def extract_pattern(start, end): #function to extract consecutive frames with the same label 
            pattern = np.array(a[start:end,:]).astype(float)
            frames = np.array(data.iloc[start:end,9]).astype(int)

            good_indices = frames != -1
            frames = frames[good_indices]
            pattern = pattern[good_indices]

            fps = 30.0
            t_sample = (frames - frames[0])/fps

            t = np.arange(int(t_sample[-1]/dt))*dt

            result = []
            for i in range(pattern.shape[1]):       
                p = np.interp(t, t_sample, pattern[:,i])
                result.append(p)
            result = np.array(result).T

            return t, result
        
        start=[]
        start.append(0)
        end=[]
        label = []
        for i in range(1, (len(labs)-1)):
            if labs[i]!=labs[i-1]:
                start.append(i)
            if labs[i]!=labs[i+1]:
                end.append(i)
                label.append(labs[i])


        t_noplay=[]
        p_noplay=[]
        t_goal=[]
        p_goal=[]
        t_aim=[]
        p_aim=[]
        
        for i in range(1,(len(start)-1)):
            if label[i]==('noplay'):
                ti, pi = extract_pattern(start[i], end[i])
                t_noplay.append(ti)
                p_noplay.append(pi)
            if label[i]==('goaloriented'):
                ti, pi = extract_pattern(start[i], end[i])
                t_goal.append(ti)
                p_goal.append(pi)
            if label[i]==('aimless'):
                ti, pi = extract_pattern(start[i], end[i])
                t_aim.append(ti)
                p_aim.append(pi)
        
############ randomly split data 80/20 into training and testing sets ############ 
        random.shuffle(p_goal)
        goal_train = p_goal[:(int(len(p_goal)*0.8))] 
        goal_test = p_goal[(int(len(p_goal)*0.8)):]
        
        random.shuffle(p_noplay)
        noplay_train = p_noplay[:(int(len(p_noplay)*0.8))]
        noplay_test = p_noplay[(int(len(p_noplay)*0.8)):]

############ create pca model and convert each pattern into single dimension (first PCA construct) ############ 
        train_all = np.vstack(goal_train+noplay_train)
        pca_model = PCA(n_components=2).fit(train_all)

        goal_patterns = {}
        i=0
        for gp in goal_train:
            goal_patterns[i] = pca_model.transform(gp)[:,0]
            i+=1

        noplay_patterns = {}
        i=0
        for npp in noplay_train:
            noplay_patterns[i] = pca_model.transform(npp)[:,0]
            i+=1

        goal_patterns = [(goal_patterns[key]) for key in goal_patterns.keys()]
        noplay_patterns = [(noplay_patterns[key]) for key in noplay_patterns.keys()] 
        
############ TRAINING WITH 80% ############ 

        theta = 1.0
        net = nengo.Network()
        with net:
            process = nengo.processes.WhiteSignal(period=100., high=1.0, y0=0)
            rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=q.n_neurons, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))
            
        s_window = int(theta/dt)
        s_pattern = len(min(goal_patterns, key=len))

        t_window = np.linspace(0, 1, s_window)
        inv_basis = rw.inverse_basis(t_window)

        eval_points=[]
        target = []
        for p in range(0,len(goal_train)):
            for i in range(s_pattern):
                eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[p], i)[:s_window]))
                target.append([1])
        
        for p in range(0,len(noplay_train)):
            for i in range(s_pattern):
                eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[p], i)[:s_window]))
                target.append([-1])
        eval_points = np.array(eval_points)
        
############ TEST ON TRAINING PATTERNS ############ 

        with net:
            result = nengo.Node(None, size_in=1)
            nengo.Connection(rw.state, result,
                             eval_points=eval_points, scale_eval_points=True,
                             function=target, synapse=0.1)
        
        train_all = np.array(goal_patterns[0])
        for gp in range(1,(len(goal_patterns))):
            train_all = np.hstack([train_all,goal_patterns[gp]])
        for npp in range(0,(len(noplay_patterns))):
            train_all = np.hstack([train_all,noplay_patterns[npp]])

        presentation_time_train=30/len(train_all)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(train_all, presentation_time=presentation_time_train))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

############ GET RESULTS ############ 

        goal_train_len = {}
        goal_train_len[0] = int(len(goal_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(goal_train)):
            goal_train_len[i] = goal_train_len[i-1]+int(len(goal_patterns[i])*presentation_time_train/dt)
        
        goal_train_means = {}
        goal_train_means[0] = np.mean(sim.data[p_result][0:goal_train_len[0]])
        for i in range(1,len(goal_train)):
            goal_train_means[i] = np.mean(sim.data[p_result][goal_train_len[i-1]:goal_train_len[i]])
            
        noplay_train_len = {}
        noplay_train_len[0] = int(len(noplay_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(noplay_train)):
            noplay_train_len[i] = noplay_train_len[i-1]+int(len(noplay_patterns[i])*presentation_time_train/dt)
        
        noplay_train_means = {}
        noplay_train_means[0] = np.mean(sim.data[p_result][goal_train_len[len(goal_train_len)-1]:noplay_train_len[0]])
        for i in range(1,len(noplay_train)):
            noplay_train_means[i] = np.mean(sim.data[p_result][noplay_train_len[i-1]:noplay_train_len[i]])
        
        for j in range(len(goal_patterns)):
            key_j = 'gtr_{}'.format(j)  
            goal_train_means[key_j] = goal_train_means.pop(j)
            
        for j in range(len(noplay_patterns)):
            key_j = 'ntr_{}'.format(j)  
            noplay_train_means[key_j] = noplay_train_means.pop(j)

############ EXTRACT 20% TEST PATTERNS ############ 
        goal_test_patterns = {}
        i=0
        for gp in goal_test:
            goal_test_patterns[i] = pca_model.transform(gp)[:,0]
            i+=1

        noplay_test_patterns = {}
        i=0
        for npp in noplay_test:
            noplay_test_patterns[i] = pca_model.transform(npp)[:,0]
            i+=1

        patterns = {}
        patterns.update(goal_test_patterns)
        patterns.update(noplay_test_patterns)

        goal_test_patterns = [(goal_test_patterns[key]) for key in goal_test_patterns.keys()]
        noplay_test_patterns = [(noplay_test_patterns[key]) for key in noplay_test_patterns.keys()] 
        
        test_all = np.array(goal_test_patterns[0])
        for gp in range(1,(len(goal_test_patterns))):
            test_all = np.hstack([test_all,goal_test_patterns[gp]])
        for npp in range(0,(len(noplay_test_patterns))):
            test_all = np.hstack([test_all,noplay_test_patterns[npp]])
            
        presentation_time_test=30/len(test_all)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(test_all, presentation_time=presentation_time_test))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

############ GET RESULTS ############ 

        goal_test_len = {}
        goal_test_len[0] = int(len(goal_test_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(goal_test)):
            goal_test_len[i] = goal_test_len[i-1]+int(len(goal_test_patterns[i])*presentation_time_train/dt)
        
        goal_test_means = {}
        goal_test_means[0] = np.mean(sim.data[p_result][0:goal_test_len[0]])
        for i in range(1,len(goal_test)):
            goal_test_means[i] = np.mean(sim.data[p_result][goal_test_len[i-1]:goal_test_len[i]])
            
        noplay_test_len = {}
        noplay_test_len[0] = int(len(noplay_test_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(noplay_test)):
            noplay_test_len[i] = noplay_test_len[i-1]+int(len(noplay_test_patterns[i])*presentation_time_train/dt)
        
        noplay_test_means = {}
        noplay_test_means[0] = np.mean(sim.data[p_result][goal_test_len[len(goal_train_len)-1]:noplay_train_len[0]])
        for i in range(1,len(noplay_test)):
            noplay_test_means[i] = np.mean(sim.data[p_result][noplay_test_len[i-1]:noplay_test_len[i]])
        
        for j in range(len(goal_test)):
            key_j = 'gtst_{}'.format(j)  
            goal_test_means[key_j] = goal_test_means.pop(j)
            
        for j in range(len(noplay_test)):
            key_j = 'ntst_{}'.format(j)  
            noplay_test_means[key_j] = noplay_test_means.pop(j)
        
    
################# TEST ON AIMLESS PATTERNS #################

        random.shuffle(p_aim)
    
        aimless_patterns = {}
        i=0
        for ap in p_aim:
            aimless_patterns[i] = pca_model.transform(ap)[:,0]
            i+=1
        
        test_aim = np.array(aimless_patterns[0])
        for ap in range(1,(len(aimless_patterns))):
            test_aim = np.hstack([test_aim,aimless_patterns[ap]])
        
        presentation_time_test=30/len(test_aim)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(test_aim, presentation_time=presentation_time_test))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

############ GET RESULTS ############ 

        aim_test_len = {}
        aim_test_len[0] = int(len(aimless_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(p_aim)):
            aim_test_len[i] = aim_test_len[i-1]+int(len(aimless_patterns[i])*presentation_time_train/dt)
        
        aim_test_means = {}
        aim_test_means[0] = np.mean(sim.data[p_result][0:aim_test_len[0]])
        for i in range(1,len(p_aim)):
            aim_test_means[i] = np.mean(sim.data[p_result][aim_test_len[i-1]:aim_test_len[i]])
            
        for j in range(len(p_aim)):
            key_j = 'aim_{}'.format(j)  
            aim_test_means[key_j] = aim_test_means.pop(j)

            
############ SAVE DATA ############ 
        
        means = {**goal_train_means, **noplay_train_means, **goal_test_means, **noplay_test_means, **aim_test_means}
            
        return means


In [3]:
PatternInterpolationTrial().run(data_dir='debug')

running PatternInterpolationTrial#20190225-140913-7100fcc1


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo\utils\numpy.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v = a[inds]
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengolib\signal\system.py:717: UserWarning: Filtering with non-SISO systems is an experimental feature that may not behave as expected.
  "expected.", UserWarning)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 1
n_neurons = 2000
aim_0 = -0.6410669297319718
aim_1 = -0.7711452249700951
aim_10 = -0.8222404224781196
aim_11 = -0.6529332395773665
aim_12 = -0.6788233599351328
aim_13 = -0.6995872032880472
aim_14 = -0.686301622430475
aim_15 = -0.7163076746486278
aim_16 = -0.7296863481846511
aim_17 = -0.722533443910825
aim_18 = -0.6631586185586502
aim_19 = -0.6301533985377806
aim_2 = -0.733610039282068
aim_20 = -0.6473733403513038
aim_3 = -0.727386925591627
aim_4 = -0.6830039449437343
aim_5 = -0.6938865875198406
aim_6 = -0.7239045176698647
aim_7 = -0.9528168581125824
aim_8 = -0.8990916497817129
aim_9 = -0.6625598832065079
gtr_0 = -0.5295081828334829
gtr_1 = -0.5911858520460955
gtr_2 = -0.6637843867954346
gtr_3 = -0.6987881443673534
gtr_4 = -0.663357882129467
gtr_5 = -0.6896657008162255
gtr_6 = -0.6903205781704083
gtr_7 = -0.6223191451557423
gtr_8 = -0.6613667942395401
gtr_9 = -0.6657699672497109
gtst_0 = -0.48435508041849107
gtst_1 = -0.4583169480399832
gtst_2 = -0.46030403111037793
ntr_0 = -0.

{'gtr_0': -0.5295081828334829,
 'gtr_1': -0.5911858520460955,
 'gtr_2': -0.6637843867954346,
 'gtr_3': -0.6987881443673534,
 'gtr_4': -0.663357882129467,
 'gtr_5': -0.6896657008162255,
 'gtr_6': -0.6903205781704083,
 'gtr_7': -0.6223191451557423,
 'gtr_8': -0.6613667942395401,
 'gtr_9': -0.6657699672497109,
 'ntr_0': -0.3783575116361502,
 'ntr_1': -0.6541413136128097,
 'ntr_2': -0.6574519907293744,
 'ntr_3': -0.5905690402755892,
 'ntr_4': -0.6522595613532081,
 'ntr_5': -0.5826361828535875,
 'ntr_6': -0.49385237130569837,
 'ntr_7': -0.573562623151967,
 'ntr_8': -0.8441520693517294,
 'ntr_9': -0.6932925090756442,
 'ntr_10': -0.709043335812544,
 'ntr_11': -0.6298474138028493,
 'ntr_12': -0.555507461534368,
 'ntr_13': -0.5580806964949063,
 'ntr_14': -0.7732577527097775,
 'ntr_15': -0.7308531367505018,
 'ntr_16': -0.6233621651623422,
 'ntr_17': -0.6505416000274552,
 'ntr_18': -0.7700067798026335,
 'ntr_19': -0.8495714703720757,
 'gtst_0': -0.48435508041849107,
 'gtst_1': -0.4583169480399832

In [ ]:
for seed in range(20):
    PatternInterpolationTrial().run(seed=seed, data_dir='experiment_childchild')

In [ ]:
### SANITY CHECK ###
from sklearn.decomposition import PCA

class PatternInterpolationTrialSanity(pytry.Trial):
    def params(self):
        self.param('number of neurons', n_neurons=2000),

        
    def evaluate(self, q): #function to fill in missing data points by interpolating
        data = pd.read_csv("pinsoro-2017-06-20-145454904791-small.csv")
        
        x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
        labs = np.array(data.iloc[:,218]).astype(str) #array of labels (purple child annotations, engagement)
        a = []
        for i in range(x.shape[1]):
            y = pd.Series(x[:,i])
            z = y.interpolate(limit_direction='both')
            a.append(z)
        a = pd.DataFrame(a)
        a = a.dropna()
        a = np.array(a).T

        dt = 0.001
        
        def extract_pattern(start, end): #function to extract consecutive frames with the same label 
            pattern = np.array(a[start:end,:]).astype(float)
            frames = np.array(data.iloc[start:end,9]).astype(int)

            good_indices = frames != -1
            frames = frames[good_indices]
            pattern = pattern[good_indices]

            fps = 30.0
            t_sample = (frames - frames[0])/fps

            t = np.arange(int(t_sample[-1]/dt))*dt

            result = []
            for i in range(pattern.shape[1]):       
                p = np.interp(t, t_sample, pattern[:,i])
                result.append(p)
            result = np.array(result).T

            return t, result
        
        start=[]
        start.append(0)
        end=[]
        label = []
        for i in range(1, (len(labs)-1)):
            if labs[i]!=labs[i-1]:
                start.append(i)
            if labs[i]!=labs[i+1]:
                end.append(i)
                label.append(labs[i])


        t_noplay=[]
        p_noplay=[]
        t_goal=[]
        p_goal=[]
        
        t1, p1 = extract_pattern(10175, 10280) 
        print('goaloriented 1 M=0.983')
        t2, p2 = extract_pattern(13811, 14182) 
        print('goaloriented 2 M=0.677')
        t3, p3 = extract_pattern(14583, 14991) 
        print('noplay 3 M=-0.195')
        t4, p4 = extract_pattern(9796, 10174) 
        print('noplay 1 M=-0.578')
        t_goal.append(t1)
        t_goal.append(t2)
        p_goal.append(p1)
        p_goal.append(p2)
        t_noplay.append(t3)
        t_noplay.append(t4)
        p_noplay.append(p3)
        p_noplay.append(p4)
        
############ randomly split data 80/20 into training and testing sets ############ 
        random.shuffle(p_goal)
        goal_train = p_goal 
        goal_test = goal_train
        
        random.shuffle(p_noplay)
        noplay_train = p_noplay
        noplay_test = noplay_train

############ create pca model and convert each pattern into single dimension (first PCA construct) ############ 
        train_all = np.vstack(goal_train+noplay_train)
        pca_model = PCA(n_components=1).fit(train_all)

        goal_patterns = {}
        i=0
        for gp in goal_train:
            goal_patterns[i] = pca_model.transform(gp)[:,0]
            i+=1

        noplay_patterns = {}
        i=0
        for npp in noplay_train:
            noplay_patterns[i] = pca_model.transform(npp)[:,0]
            i+=1

        goal_patterns = [(goal_patterns[key]) for key in goal_patterns.keys()]
        noplay_patterns = [(noplay_patterns[key]) for key in noplay_patterns.keys()] 
        
############ TRAINING WITH 80% ############ 

        theta = 1.0
        net = nengo.Network()
        with net:
            process = nengo.processes.WhiteSignal(period=100., high=1.0, y0=0)
            rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=q.n_neurons, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))
            
        s_window = int(theta/dt)
        s_pattern = len(min(goal_patterns, key=len))

        t_window = np.linspace(0, 1, s_window)
        inv_basis = rw.inverse_basis(t_window)

        eval_points=[]
        target = []
        for p in range(0,len(goal_train)):
            for i in range(s_pattern):
                eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[p], i)[:s_window]))
                target.append([1])
        
        for p in range(0,len(noplay_train)):
            for i in range(s_pattern):
                eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[p], i)[:s_window]))
                target.append([-1])
        eval_points = np.array(eval_points)
        
############ TEST ON TRAINING PATTERNS ############ 

        with net:
            result = nengo.Node(None, size_in=1)
            nengo.Connection(rw.state, result,
                             eval_points=eval_points, scale_eval_points=True,
                             function=target, synapse=0.1)
        
        train_all = np.array(goal_patterns[0])
        for gp in range(1,(len(goal_patterns))):
            train_all = np.hstack([train_all,goal_patterns[gp]])
        for npp in range(0,(len(noplay_patterns))):
            train_all = np.hstack([train_all,noplay_patterns[npp]])

        presentation_time_train=30/len(train_all)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(train_all, presentation_time=presentation_time_train))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

############ GET RESULTS ############ 

        goal_train_len = {}
        goal_train_len[0] = int(len(goal_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(goal_train)):
            goal_train_len[i] = goal_train_len[i-1]+int(len(goal_patterns[i])*presentation_time_train/dt)
        
        goal_train_means = {}
        goal_train_means[0] = np.mean(sim.data[p_result][0:goal_train_len[0]])
        for i in range(1,len(goal_train)):
            goal_train_means[i] = np.mean(sim.data[p_result][goal_train_len[i-1]:goal_train_len[i]])
            
        noplay_train_len = {}
        noplay_train_len[0] = int(len(noplay_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(noplay_train)):
            noplay_train_len[i] = noplay_train_len[i-1]+int(len(noplay_patterns[i])*presentation_time_train/dt)
        
        noplay_train_means = {}
        noplay_train_means[0] = np.mean(sim.data[p_result][0:noplay_train_len[0]])
        for i in range(1,len(noplay_train)):
            noplay_train_means[i] = np.mean(sim.data[p_result][noplay_train_len[i-1]:noplay_train_len[i]])
        
        for j in range(len(goal_patterns)):
            key_j = 'gtr_{}'.format(j)  
            goal_train_means[key_j] = goal_train_means.pop(j)
            
        for j in range(len(noplay_patterns)):
            key_j = 'ntr_{}'.format(j)  
            noplay_train_means[key_j] = noplay_train_means.pop(j)

############ EXTRACT 20% TEST PATTERNS ############ 
        goal_test_patterns = {}
        i=0
        for gp in goal_test:
            goal_test_patterns[i] = pca_model.transform(gp)[:,0]
            i+=1

        noplay_test_patterns = {}
        i=0
        for npp in noplay_test:
            noplay_test_patterns[i] = pca_model.transform(npp)[:,0]
            i+=1

        patterns = {}
        patterns.update(goal_test_patterns)
        patterns.update(noplay_test_patterns)

        goal_test_patterns = [(goal_test_patterns[key]) for key in goal_test_patterns.keys()]
        noplay_test_patterns = [(noplay_test_patterns[key]) for key in noplay_test_patterns.keys()] 
        
        test_all = np.array(goal_test_patterns[0])
        for gp in range(1,(len(goal_test_patterns))):
            test_all = np.hstack([test_all,goal_test_patterns[gp]])
        for npp in range(0,(len(noplay_test_patterns))):
            test_all = np.hstack([test_all,noplay_test_patterns[npp]])
            
        presentation_time_test=30/len(test_all)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(test_all, presentation_time=presentation_time_test))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

############ GET RESULTS ############ 

        goal_test_len = {}
        goal_test_len[0] = int(len(goal_test_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(goal_test)):
            goal_test_len[i] = goal_test_len[i-1]+int(len(goal_test_patterns[i])*presentation_time_train/dt)
        
        goal_test_means = {}
        goal_test_means[0] = np.mean(sim.data[p_result][0:goal_test_len[0]])
        for i in range(1,len(goal_test)):
            goal_test_means[i] = np.mean(sim.data[p_result][goal_test_len[i-1]:goal_test_len[i]])
            
        noplay_test_len = {}
        noplay_test_len[0] = int(len(noplay_test_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(noplay_test)):
            noplay_test_len[i] = noplay_test_len[i-1]+int(len(noplay_test_patterns[i])*presentation_time_train/dt)
        
        noplay_test_means = {}
        noplay_test_means[0] = np.mean(sim.data[p_result][0:noplay_train_len[0]])
        for i in range(1,len(noplay_test)):
            noplay_test_means[i] = np.mean(sim.data[p_result][noplay_test_len[i-1]:noplay_test_len[i]])
        
        for j in range(len(goal_test)):
            key_j = 'gtst_{}'.format(j)  
            goal_test_means[key_j] = goal_test_means.pop(j)
            
        for j in range(len(noplay_test)):
            key_j = 'ntst_{}'.format(j)  
            noplay_test_means[key_j] = noplay_test_means.pop(j)
        
############ SAVE DATA ############ 
        
        means = {**goal_train_means, **noplay_train_means, **goal_test_means, **noplay_test_means}
            
        return means

In [ ]:
PatternInterpolationTrialSanity().run(data_dir='sanity-check')

In [8]:
data = pd.read_csv("pinsoro-2017-06-20-145454904791-small.csv", low_memory=False)

x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
labs = np.array(data.iloc[:,218]).astype(str) #array of labels (purple child annotations, engagement)
a = []
for i in range(x.shape[1]):
    y = pd.Series(x[:,i])
    z = y.interpolate(limit_direction='both')
    a.append(z)
a = pd.DataFrame(a)
a = a.dropna()
a = np.array(a).T

dt = 0.001

def extract_pattern(start, end): #function to extract consecutive frames with the same label 
    pattern = np.array(a[start:end,:]).astype(float)
    frames = np.array(data.iloc[start:end,9]).astype(int)

    good_indices = frames != -1
    frames = frames[good_indices]
    pattern = pattern[good_indices]

    fps = 30.0
    t_sample = (frames - frames[0])/fps

    t = np.arange(int(t_sample[-1]/dt))*dt

    result = []
    for i in range(pattern.shape[1]):       
        p = np.interp(t, t_sample, pattern[:,i])
        result.append(p)
    result = np.array(result).T

    return t, result

start=[]
start.append(0)
end=[]
label = []
for i in range(1, (len(labs)-1)):
    if labs[i]!=labs[i-1]:
        start.append(i)
    if labs[i]!=labs[i+1]:
        end.append(i)
        label.append(labs[i])


t_noplay=[]
p_noplay=[]
t_goal=[]
p_goal=[]
t_aim=[]
p_aim=[]

for i in range(1,(len(start)-1)):
    if label[i]==('noplay'):
        ti, pi = extract_pattern(start[i], end[i])
        t_noplay.append(ti)
        p_noplay.append(pi)
    if label[i]==('goaloriented'):
        ti, pi = extract_pattern(start[i], end[i])
        t_goal.append(ti)
        p_goal.append(pi)
    if label[i]==('aimless'):
        ti, pi = extract_pattern(start[i], end[i])
        t_aim.append(ti)
        p_aim.append(pi)

############ randomly split data 80/20 into training and testing sets ############ 
random.shuffle(p_goal)
goal_train = p_goal[:(int(len(p_goal)*0.8))] 
goal_test = p_goal[(int(len(p_goal)*0.8)):]

random.shuffle(p_noplay)
noplay_train = p_noplay[:(int(len(p_noplay)*0.8))]
noplay_test = p_noplay[(int(len(p_noplay)*0.8)):]

############ create pca model and convert each pattern into single dimension (first PCA construct) ############ 
train_all = np.vstack(goal_train+noplay_train)
pca_model = PCA(n_components=2).fit(train_all)

goal_patterns = {}
i=0
for gp in goal_train:
    goal_patterns[i] = pca_model.transform(gp)[:,0]
    i+=1

noplay_patterns = {}
i=0
for npp in noplay_train:
    noplay_patterns[i] = pca_model.transform(npp)[:,0]
    i+=1

goal_patterns = [(goal_patterns[key]) for key in goal_patterns.keys()]
noplay_patterns = [(noplay_patterns[key]) for key in noplay_patterns.keys()] 

############ TRAINING WITH 80% ############ 

theta = 1.0
net = nengo.Network()
with net:
    process = nengo.processes.WhiteSignal(period=100., high=1.0, y0=0)
    rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))

s_window = int(theta/dt)
s_pattern = len(min(goal_patterns, key=len))

t_window = np.linspace(0, 1, s_window)
inv_basis = rw.inverse_basis(t_window)

eval_points=[]
target = []
for p in range(0,len(goal_train)):
    for i in range(s_pattern):
        eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[p], i)[:s_window]))
        target.append([1])

for p in range(0,len(noplay_train)):
    for i in range(s_pattern):
        eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[p], i)[:s_window]))
        target.append([-1])
eval_points = np.array(eval_points)

############ TEST ON TRAINING PATTERNS ############ 

with net:
    result = nengo.Node(None, size_in=1)
    nengo.Connection(rw.state, result,
                     eval_points=eval_points, scale_eval_points=True,
                     function=target, synapse=0.1)

train_all = np.array(goal_patterns[0])
print(train_all)
print(train_all.shape)
for gp in range(1,(len(goal_patterns))):
    train_all = np.hstack([train_all,goal_patterns[gp]])
for npp in range(0,(len(noplay_patterns))):
    train_all = np.hstack([train_all,noplay_patterns[npp]])
print(train_all)
print(train_all.shape)

C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)


[-2.41043076 -2.41044708 -2.41046339 ...  3.64750141  3.64741604
  3.64733068]
(20833,)
[-2.41043076 -2.41044708 -2.41046339 ... -2.27615761 -2.27638832
 -2.27661904]
(557487,)


In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score

import pytry

from sklearn.decomposition import PCA

data = pd.read_csv("pinsoro-2017-06-20-145454904791-small.csv", low_memory=False)
        
x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
labs = np.array(data.iloc[:,218]).astype(str) #array of labels (purple child annotations, engagement)
a = []
for i in range(x.shape[1]):
    y = pd.Series(x[:,i])
    z = y.interpolate(limit_direction='both')
    a.append(z)
a = pd.DataFrame(a)
a = a.dropna()
a = np.array(a).T

dt = 0.001

def extract_pattern(start, end): #function to extract consecutive frames with the same label 
    pattern = np.array(a[start:end,:]).astype(float)
    frames = np.array(data.iloc[start:end,9]).astype(int)

    good_indices = frames != -1
    frames = frames[good_indices]
    pattern = pattern[good_indices]

    fps = 30.0
    t_sample = (frames - frames[0])/fps

    t = np.arange(int(t_sample[-1]/dt))*dt

    result = []
    for i in range(pattern.shape[1]):       
        p = np.interp(t, t_sample, pattern[:,i])
        result.append(p)
    result = np.array(result).T

    return t, result

start=[]
start.append(0)
end=[]
label = []
for i in range(1, (len(labs)-1)):
    if labs[i]!=labs[i-1]:
        start.append(i)
    if labs[i]!=labs[i+1]:
        end.append(i)
        label.append(labs[i])


t_noplay=[]
p_noplay=[]
t_goal=[]
p_goal=[]
t_aim=[]
p_aim=[]

for i in range(1,(len(start)-1)):
    if label[i]==('noplay'):
        ti, pi = extract_pattern(start[i], end[i])
        t_noplay.append(ti)
        p_noplay.append(pi)
    if label[i]==('goaloriented'):
        ti, pi = extract_pattern(start[i], end[i])
        t_goal.append(ti)
        p_goal.append(pi)
    if label[i]==('aimless'):
        ti, pi = extract_pattern(start[i], end[i])
        t_aim.append(ti)
        p_aim.append(pi)

############ randomly split data 80/20 into training and testing sets ############ 
random.shuffle(p_goal)
goal_train = p_goal[:(int(len(p_goal)*0.8))] 
goal_test = p_goal[(int(len(p_goal)*0.8)):]

random.shuffle(p_noplay)
noplay_train = p_noplay[:(int(len(p_noplay)*0.8))]
noplay_test = p_noplay[(int(len(p_noplay)*0.8)):]

############ create pca model and convert each pattern into single dimension (first PCA construct) ############ 
train_all = np.vstack(goal_train+noplay_train)
pca_model = PCA(n_components=2).fit(train_all)

goal_patterns = {}
i=0
for gp in goal_train:
    goal_patterns[i] = pca_model.transform(gp)[:,0]
    i+=1

noplay_patterns = {}
i=0
for npp in noplay_train:
    noplay_patterns[i] = pca_model.transform(npp)[:,0]
    i+=1

goal_patterns = [(goal_patterns[key]) for key in goal_patterns.keys()]
noplay_patterns = [(noplay_patterns[key]) for key in noplay_patterns.keys()] 

############ TRAINING WITH 80% ############ 

theta = 1.0
net = nengo.Network()
with net:
    process = nengo.processes.WhiteSignal(period=100., high=1.0, y0=0)
    rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))

s_window = int(theta/dt)
s_pattern = len(min(goal_patterns, key=len))

t_window = np.linspace(0, 1, s_window)
inv_basis = rw.inverse_basis(t_window)

eval_points=[]
target = []
for p in range(0,len(goal_train)):
    for i in range(s_pattern):
        eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[p], i)[:s_window]))
        target.append([1])

for p in range(0,len(noplay_train)):
    for i in range(s_pattern):
        eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[p], i)[:s_window]))
        target.append([-1])
eval_points = np.array(eval_points)

############ TEST ON TRAINING PATTERNS ############ 

with net:
    result = nengo.Node(None, size_in=1)
    nengo.Connection(rw.state, result,
                     eval_points=eval_points, scale_eval_points=True,
                     function=target, synapse=0.1)

train_all = np.array(goal_patterns[0])
for gp in range(1,(len(goal_patterns))):
    train_all = np.hstack([train_all,goal_patterns[gp]])
for npp in range(0,(len(noplay_patterns))):
    train_all = np.hstack([train_all,noplay_patterns[npp]])

presentation_time_train=30/len(train_all)

model = nengo.Network()
model.networks.append(net)
with model:    
    stim = nengo.Node(nengo.processes.PresentInput(train_all, presentation_time=presentation_time_train))
    nengo.Connection(stim, rw.input, synapse=None)

    p_result = nengo.Probe(result)
    p_stim = nengo.Probe(stim)
sim = nengo.Simulator(model)
sim.run(30)

############ GET RESULTS ############ 
print('goal_patterns')
print(goal_patterns)
print(goal_patterns.shape)

goal_train_len = {}
goal_train_len[0] = int(len(goal_patterns[0])*presentation_time_train/dt)
for i in range(1,len(goal_train)):
    goal_train_len[i] = goal_train_len[i-1]+int(len(goal_patterns[i])*presentation_time_train/dt)

goal_train_means = {}
goal_train_means[0] = np.mean(sim.data[p_result][0:goal_train_len[0]])
for i in range(1,len(goal_train)):
    goal_train_means[i] = np.mean(sim.data[p_result][goal_train_len[i-1]:goal_train_len[i]])

noplay_train_len = {}
noplay_train_len[0] = int(len(noplay_patterns[0])*presentation_time_train/dt)
for i in range(1,len(noplay_train)):
    noplay_train_len[i] = noplay_train_len[i-1]+int(len(noplay_patterns[i])*presentation_time_train/dt)

noplay_train_means = {}
noplay_train_means[0] = np.mean(sim.data[p_result][0:noplay_train_len[0]])
for i in range(1,len(noplay_train)):
    noplay_train_means[i] = np.mean(sim.data[p_result][noplay_train_len[i-1]:noplay_train_len[i]])

for j in range(len(goal_patterns)):
    key_j = 'gtr_{}'.format(j)  
    goal_train_means[key_j] = goal_train_means.pop(j)

for j in range(len(noplay_patterns)):
    key_j = 'ntr_{}'.format(j)  
    noplay_train_means[key_j] = noplay_train_means.pop(j)

/home/maddy/.local/lib/python3.6/site-packages/scipy/signal/filter_design.py:1619: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
/home/maddy/.local/lib/python3.6/site-packages/nengo/utils/numpy.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v = a[inds]
/home/maddy/.local/lib/python3.6/site-packages/nengolib/signal/system.py:717: UserWarning: Filtering with non-SISO systems is an experimental feature that may not behave as expected.
  "expected.", UserWarning)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

goal_patterns
[array([-1.71990005, -1.71964754, -1.71939502, ..., -1.55726674,
       -1.55721185, -1.55715697]), array([-1.61755399, -1.61783406, -1.61811413, ..., -1.68581513,
       -1.68586919, -1.68592325]), array([-1.57645914, -1.57715108, -1.57784302, ...,  4.36734316,
        4.36714561,  4.36694806]), array([-1.68490215, -1.68502844, -1.68515473, ..., -1.45161597,
       -1.45176773, -1.45191949]), array([4.19672416, 4.19657595, 4.19642775, ..., 4.29133234, 4.29203627,
       4.29274019]), array([4.00720371, 4.006889  , 4.0065743 , ..., 4.11471821, 4.1148744 ,
       4.11503059]), array([ 4.42098077,  4.42083733,  4.4206939 , ..., -1.6741372 ,
       -1.67422892, -1.67432064]), array([-1.76834237, -1.7686831 , -1.76902383, ...,  3.87793804,
        3.87791456,  3.87789108]), array([-1.90754061, -1.90778452, -1.90802843, ..., -1.76859993,
       -1.76879649, -1.76899305]), array([-0.91251198, -0.91266556, -0.91281915, ...,  4.18603452,
        4.18580373,  4.18557294])]


AttributeError: 'list' object has no attribute 'shape'

In [7]:
print(len(goal_patterns[0]))
print(len((goal_patterns[0]*presentation_time_train/dt)))
print(presentation_time_train/dt)
f = len(goal_patterns[0])*presentation_time_train/dt
print(f)
print(len(sim.data[p_result]))
print(len(train_all))

38866
38866
0.04316621294180619
1677.6980321962394
30000
694988


In [8]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
